In [1]:
import pandas as pd
import numpy as np

In [2]:
files = {
    "S&P_500_2010_2023": "../data/S&P_500_2010_2023.csv",
    "spy_2020_2022": "../data/spy_2020_2022.csv",
    "SPY_Options": "../data/SPY_Options.csv",

    # AAPL split files
    "AAPL_2016_2020": "../data/aapl_2016_2020.csv",
    "AAPL_2021_2023": "../data/aapl_2021_2023.csv"
}

In [3]:
datasets = {}

for name, path in files.items():
    df = pd.read_csv(path, low_memory=False)
    df.columns = (
    df.columns
      .str.strip()
      .str.replace("[", "", regex=False)
      .str.replace("]", "", regex=False)
      .str.upper()
    )

    # Standardize column names if needed
    if "QUOTE_DATE" in df.columns:
        df["QUOTE_DATE"] = pd.to_datetime(
            df["QUOTE_DATE"],
            format="mixed",
            dayfirst=True,
            errors="coerce"
        )
    elif "Date" in df.columns:
        df["QUOTE_DATE"] = pd.to_datetime(
            df["Date"],
            format="mixed",
            dayfirst=True,
            errors="coerce"
        )
    else:
        raise ValueError(f"{name}: No quote date column found")

    if "EXPIRE_DATE" in df.columns:
        df["EXPIRE_DATE"] = pd.to_datetime(
            df["EXPIRE_DATE"],
            format="mixed",
            dayfirst=True,
            errors="coerce"
        )
    else:
        df["EXPIRE_DATE"] = pd.NaT  # some datasets may not have expiry

    df = df.dropna(subset=["QUOTE_DATE"])

    datasets[name] = df

    print(f"{name}: loaded {len(df):,} rows")


S&P_500_2010_2023: loaded 1,048,531 rows
spy_2020_2022: loaded 3,589,079 rows
SPY_Options: loaded 4,195,810 rows
AAPL_2016_2020: loaded 1,015,352 rows
AAPL_2021_2023: loaded 548,163 rows


In [4]:
# ============================================================
# Merge AAPL files into a single continuous dataset
# ============================================================

aapl_16_20 = datasets["AAPL_2016_2020"].copy()
aapl_21_23 = datasets["AAPL_2021_2023"].copy()

aapl = pd.concat([aapl_16_20, aapl_21_23], axis=0)

aapl = (
    aapl
    .dropna(subset=["QUOTE_DATE"])
    .sort_values("QUOTE_DATE")
    .reset_index(drop=True)
)

datasets["AAPL_FULL_2016_2023"] = aapl

print("AAPL merged shape:", aapl.shape)
print("AAPL date range:",
      aapl["QUOTE_DATE"].min(),
      "→",
      aapl["QUOTE_DATE"].max())

AAPL merged shape: (1563515, 33)
AAPL date range: 2016-01-04 00:00:00 → 2023-03-31 00:00:00


In [5]:
summary = []

for name, df in datasets.items():
    summary.append({
        "Dataset": name,
        "Rows": len(df),
        "Quote min": df["QUOTE_DATE"].min(),
        "Quote max": df["QUOTE_DATE"].max(),
        "Unique QUOTE_DATE": df["QUOTE_DATE"].nunique(),
        "Unique EXPIRE_DATE": df["EXPIRE_DATE"].nunique() if "EXPIRE_DATE" in df else "N/A"
    })

summary_df = pd.DataFrame(summary).set_index("Dataset")
summary_df

,Rows,Quote min,Quote max,Unique QUOTE_DATE,Unique EXPIRE_DATE
Dataset,,,,,
S&P_500_2010_2023,1048531,2010-01-04,2013-08-16,903,219
spy_2020_2022,3589079,2020-01-02,2022-12-30,758,514
SPY_Options,4195810,2010-01-04,2023-12-29,3500,1572
AAPL_2016_2020,1015352,2016-01-04,2020-12-31,1253,280
AAPL_2021_2023,548163,2021-01-04,2023-03-31,570,139
AAPL_FULL_2016_2023,1563515,2016-01-04,2023-03-31,1823,402


In [6]:
year_table = {}

for name, df in datasets.items():
    year_table[name] = (
        df["QUOTE_DATE"]
        .dt.year
        .value_counts()
        .sort_index()
    )

year_df = pd.concat(year_table, axis=1).fillna(0).astype(int)
year_df

,S&P_500_2010_2023,spy_2020_2022,SPY_Options,AAPL_2016_2020,AAPL_2021_2023,AAPL_FULL_2016_2023
QUOTE_DATE,,,,,,
2010,233248,0,149551,0,0,0
2011,248382,0,149239,0,0,0
2012,324606,0,160530,0,0,0
2013,242295,0,64505,0,0,0
2020,0,1164401,682266,292748,0,292748
2021,0,1277698,777184,0,239822,239822
2022,0,1146980,660244,0,236004,236004
2014,0,0,68439,0,0,0
2015,0,0,90455,0,0,0


In [7]:
month_table = {}

for name, df in datasets.items():
    month_table[name] = (
        df["QUOTE_DATE"]
        .dt.to_period("M")
        .value_counts()
        .sort_index()
    )

month_df = pd.concat(month_table, axis=1).fillna(0).astype(int)
month_df.head(24)   # first 2 years

,S&P_500_2010_2023,spy_2020_2022,SPY_Options,AAPL_2016_2020,AAPL_2021_2023,AAPL_FULL_2016_2023
QUOTE_DATE,,,,,,
2010-01,17572,0,12283,0,0,0
2010-02,17349,0,9156,0,0,0
2010-03,20756,0,13233,0,0,0
2010-04,19107,0,11727,0,0,0
2010-05,17947,0,11574,0,0,0
2010-06,20873,0,14333,0,0,0
2010-07,19084,0,12699,0,0,0
2010-08,20938,0,13570,0,0,0
2010-09,20646,0,13050,0,0,0


In [8]:
gap_summary = []

for name, df in datasets.items():
    dates = pd.Series(df["QUOTE_DATE"].dt.normalize().unique()).sort_values()
    full_range = pd.date_range(dates.min(), dates.max(), freq="B")
    missing = len(full_range.difference(dates))

    gap_summary.append({
        "Dataset": name,
        "Expected business days": len(full_range),
        "Actual quote days": len(dates),
        "Missing days": missing
    })

pd.DataFrame(gap_summary).set_index("Dataset")

,Expected business days,Actual quote days,Missing days
Dataset,,,
S&P_500_2010_2023,945,903,42
spy_2020_2022,782,758,24
SPY_Options,3650,3500,150
AAPL_2016_2020,1304,1253,51
AAPL_2021_2023,585,570,15
AAPL_FULL_2016_2023,1890,1823,67


In [9]:
expiry_stats = []

for name, df in datasets.items():
    if df["EXPIRE_DATE"].notna().sum() == 0:
        continue

    dte = (df["EXPIRE_DATE"] - df["QUOTE_DATE"]).dt.days

    expiry_stats.append({
        "Dataset": name,
        "Min DTE": dte.min(),
        "Median DTE": dte.median(),
        "Max DTE": dte.max(),
        "Unique expiries": df["EXPIRE_DATE"].nunique()
    })

pd.DataFrame(expiry_stats).set_index("Dataset")

,Min DTE,Median DTE,Max DTE,Unique expiries
Dataset,,,,
S&P_500_2010_2023,0,76.0,1093,219
spy_2020_2022,0,64.0,1096,514
SPY_Options,0,101.0,1096,1572
AAPL_2016_2020,0,64.0,891,280
AAPL_2021_2023,0,101.0,1059,139
AAPL_FULL_2016_2023,0,74.0,1059,402


In [10]:
verdicts = []

for name, df in datasets.items():
    years = df["QUOTE_DATE"].dt.year.unique()

    verdicts.append({
        "Dataset": name,
        "Covers 2010+": years.min() <= 2010,
        "Covers 2023": years.max() >= 2023,
        "Years covered": len(years),
        "Suitable for full study": (years.min() <= 2010 and years.max() >= 2023)
    })

pd.DataFrame(verdicts).set_index("Dataset")

,Covers 2010+,Covers 2023,Years covered,Suitable for full study
Dataset,,,,
S&P_500_2010_2023,True,False,4,False
spy_2020_2022,False,False,3,False
SPY_Options,True,True,14,True
AAPL_2016_2020,False,False,5,False
AAPL_2021_2023,False,True,3,False
AAPL_FULL_2016_2023,False,True,8,False


In [ ]:
import pandas as pd

try:
    # Read the CSV file (replace 'your_file.csv' with your actual file path)
    df = pd.read_csv('aapl_2016_2020.csv')
    
    # Print only the column titles
    print("Column Titles:")
    for col in df.columns:
        print(col)